 # Install/Import Packages and Download/Unzip Dataset

 ---

 The following code installs d3rlpy, imports all necessary dependencies, and downloads/unzips the dataset from Zenodo.

In [ ]:
!pip install d3rlpy==2.6.1 gymnasium==0.29.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 34.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.8 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827622 sha256=2476474435436d494fc1a4df932704fb6922daab621855bcf5715cf883e5a874
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not cur

In [ ]:
from google.colab import files
import numpy as np
import d3rlpy
import os

In [ ]:
!wget -O full_dataset.zip "https://zenodo.org/record/13830810/files/full_dataset.zip?download=1"

--2024-09-25 20:18:33--  https://zenodo.org/record/13830810/files/full_dataset.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.98.238, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/13830810/files/full_dataset.zip [following]
--2024-09-25 20:18:34--  https://zenodo.org/records/13830810/files/full_dataset.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 2161582746 (2.0G) [application/octet-stream]
Saving to: ‘full_dataset.zip’

full_dataset.zip    100%[===================>]   2.01G  15.3MB/s    in 2m 16s  

2024-09-25 20:20:49 (15.2 MB/s) - ‘full_dataset.zip’ saved [2161582746/2161582746]



In [ ]:
!unzip full_dataset.zip -d /content/
data_dir = '/content/content/data/data'

Streaming output truncated to the last 5000 lines.
  inflating: /content/content/data/data/ep_38406.npz  
  inflating: /content/content/data/data/ep_45886.npz  
  inflating: /content/content/data/data/ep_80720.npz  
  inflating: /content/content/data/data/ep_41811.npz  
  inflating: /content/content/data/data/ep_74361.npz  
  inflating: /content/content/data/data/ep_36142.npz  
  inflating: /content/content/data/data/ep_92647.npz  
  inflating: /content/content/data/data/ep_35769.npz  
  inflating: /content/content/data/data/ep_70452.npz  
  inflating: /content/content/data/data/ep_89001.npz  
  inflating: /content/content/data/data/ep_92324.npz  
  inflating: /content/content/data/data/ep_21612.npz  
  inflating: /content/content/data/data/ep_51788.npz  
  inflating: /content/content/data/data/ep_51083.npz  
  inflating: /content/content/data/data/ep_54442.npz  
  inflating: /content/content/data/data/ep_97628.npz  
  inflating: /content/content/data/data/ep_26720.npz  
  inflating: /

# Prepare Dataset for d3rlpy

---

The following code loads all of the data into arrays and creates a d3rlpy dataset.

In [ ]:
total_steps = 5000000

all_observations = np.empty((total_steps, 3, 64, 64), dtype=np.uint8)
all_actions = np.empty((total_steps, 3), dtype=np.float32)
all_rewards = np.empty((total_steps,), dtype=np.float32)
all_terminals = np.empty((total_steps,), dtype=bool)

current_index = 0

for i in range(1, 50001):
    episode_file = os.path.join(data_dir, f'ep_{i}.npz')
    episode_data = np.load(episode_file)

    num_steps = episode_data['terminals'].shape[0]

    all_observations[current_index:current_index + num_steps] = np.transpose(episode_data['observations'], (0, 3, 1, 2))
    all_actions[current_index:current_index + num_steps] = episode_data['actions']
    all_rewards[current_index:current_index + num_steps] = episode_data['rewards']
    all_terminals[current_index:current_index + num_steps] = episode_data['terminals']

    current_index += num_steps

all_observations = all_observations[:current_index]
all_actions = all_actions[:current_index]
all_rewards = all_rewards[:current_index]
all_terminals = all_terminals[:current_index]

In [ ]:
dataset = d3rlpy.dataset.MDPDataset(
    observations=all_observations,
    actions=all_actions,
    rewards=all_rewards,
    terminals=all_terminals,
)

2024-09-25 20:30.19 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(3,)]) observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 64, 64)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2024-09-25 20:30.19 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2024-09-25 20:30.19 [info     ] Action size has been automatically determined. action_size=3


# Model Training

---

The following code trains a behavior cloning algorithm for 50 epochs at 10000 steps per epoch.

In [ ]:
BC = d3rlpy.algos.BCConfig(observation_scaler=d3rlpy.preprocessing.PixelObservationScaler(),).create(device='cuda')
BC.fit(dataset=dataset, n_steps=500000, n_steps_per_epoch=10000)

2024-09-25 20:30.21 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 64, 64)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(3,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=3)
2024-09-25 20:30.21 [info     ] Directory is created at d3rlpy_logs/BC_20240925203021
2024-09-25 20:30.21 [debug    ] Building models...            
2024-09-25 20:30.23 [debug    ] Models have been built.       
2024-09-25 20:30.23 [info     ] Parameters                     params={'observation_shape': [3, 64, 64], 'action_size': 3, 'config': {'type': 'bc', 'params': {'batch_size': 100, 'gamma': 0.99, 'observation_scaler': {'type': 'pixel', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'learning_rate': 0.001, 'policy_type': 'deterministic', 'optim_factory': {'type': '

Epoch 1/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 20:31.45 [info     ] BC_20240925203021: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.0028868495225906374, 'time_algorithm_update': 0.0051183199644088745, 'loss': 0.015720033557107674, 'time_step': 0.008097256326675415} step=10000
2024-09-25 20:31.45 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925203021/model_10000.d3


Epoch 2/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 20:33.05 [info     ] BC_20240925203021: epoch=2 step=20000 epoch=2 metrics={'time_sample_batch': 0.0028759174346923828, 'time_algorithm_update': 0.004972385549545288, 'loss': 0.009199370395438746, 'time_step': 0.007936518120765686} step=20000
2024-09-25 20:33.05 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925203021/model_20000.d3


...

Epoch 49/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 21:36.04 [info     ] BC_20240925203021: epoch=49 step=490000 epoch=49 metrics={'time_sample_batch': 0.0028960471630096437, 'time_algorithm_update': 0.005007071328163147, 'loss': 0.0052135687610134485, 'time_step': 0.007993524765968324} step=490000
2024-09-25 21:36.04 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925203021/model_490000.d3


Epoch 50/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 21:37.25 [info     ] BC_20240925203021: epoch=50 step=500000 epoch=50 metrics={'time_sample_batch': 0.002898890233039856, 'time_algorithm_update': 0.005013119626045227, 'loss': 0.005187341761682182, 'time_step': 0.00800225706100464} step=500000
2024-09-25 21:37.25 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925203021/model_500000.d3


[(1,
  {'time_sample_batch': 0.0028868495225906374,
   'time_algorithm_update': 0.0051183199644088745,
   'loss': 0.015720033557107674,
   'time_step': 0.008097256326675415}),
 (2,
  {'time_sample_batch': 0.0028759174346923828,
   'time_algorithm_update': 0.004972385549545288,
   'loss': 0.009199370395438746,
   'time_step': 0.007936518120765686}),
 (3,
  {'time_sample_batch': 0.0028751471042633057,
   'time_algorithm_update': 0.004967793107032776,
   'loss': 0.008171565019059926,
   'time_step': 0.007931830477714539}),
 (4,
  {'time_sample_batch': 0.002876189041137695,
   'time_algorithm_update': 0.004966652631759644,
   'loss': 0.007651704471884295,
   'time_step': 0.007932533311843872}),
 (5,
  {'time_sample_batch': 0.002893471908569336,
   'time_algorithm_update': 0.004987011528015136,
   'loss': 0.007258199187950231,
   'time_step': 0.00796983814239502}),
 (6,
  {'time_sample_batch': 0.0028774746417999267,
   'time_algorithm_update': 0.004984899401664734,
   'loss': 0.007014143728

In [ ]:
!zip -r model_half_dataset.zip d3rlpy_logs

  adding: d3rlpy_logs/ (stored 0%)
  adding: d3rlpy_logs/BC_20240925203021/ (stored 0%)
  adding: d3rlpy_logs/BC_20240925203021/model_90000.d3 (deflated 35%)
  adding: d3rlpy_logs/BC_20240925203021/time_algorithm_update.csv (deflated 61%)
  adding: d3rlpy_logs/BC_20240925203021/time_step.csv (deflated 61%)
  adding: d3rlpy_logs/BC_20240925203021/model_30000.d3 (deflated 22%)
  adding: d3rlpy_logs/BC_20240925203021/model_320000.d3 (deflated 39%)
  adding: d3rlpy_logs/BC_20240925203021/model_440000.d3 (deflated 40%)
  adding: d3rlpy_logs/BC_20240925203021/model_340000.d3 (deflated 39%)
  adding: d3rlpy_logs/BC_20240925203021/model_250000.d3 (deflated 40%)
  adding: d3rlpy_logs/BC_20240925203021/model_160000.d3 (deflated 40%)
  adding: d3rlpy_logs/BC_20240925203021/model_290000.d3 (deflated 40%)
  adding: d3rlpy_logs/BC_20240925203021/model_130000.d3 (deflated 39%)
  adding: d3rlpy_logs/BC_20240925203021/model_500000.d3 (deflated 39%)
  adding: d3rlpy_logs/BC_20240925203021/model_140000.d